# Multi-Layer Perceptron

Just to get familiar with the tool, we're going to be making something a bit more basic here.

In [1]:
import mlx.core as mlx
import mlx.nn as nn
import mlx.optimizers as optim

import numpy as np

# Initializers

According to what's next here, the model is defined as the MLP class which inherits from `mlx.nn.Module`. We follow the standard practice to make a new model.

1. Define an `__init__` where the parameters and/or submodules are setup.
2. Define a `__call__` where the computationo is implemented.

In [2]:
class MLP(nn.Module):
    def __init__(
            self,
            num_layers: int,
            input_dim: int,
            hidden_dim: int,
            output_dim: int
    ):
        super().__init__()
        layer_sizes = [input_dim] + [hidden_dim] * num_layers + [output_dim]
        self.layers = [
            nn.Linear(idim, odim)
            for idim, odim in zip(layer_sizes[:-1], layer_sizes[1:])
        ]

    def __call__( self, x ):
        for l in self.layers[:-1]:
            x = mlx.maximum(l(x), 0.0)
        return self.layers[-1](x)

In [3]:
def loss_fn(model, X, y):
    return mlx.mean(nn.losses.cross_entropy(model(X), y))

In [4]:
def eval_fn(model, X, y):
    return mlx.mean(mlx.argmax(model(X), axis=1) == y)

In [5]:
num_layers = 2
hidden_dim = 32
num_classes = 10
batch_size = 256
num_epochs = 10
learning_rate = 1e-1

# Load the data
import mnist
train_images, train_labels, test_images, test_labels = map(
    mlx.array, mnist.mnist()
)

In [6]:
def batch_iterate(batch_size, X, y):
    perm = mlx.array(np.random.permutation(y.size))
    for s in range(0, y.size, batch_size):
        ids = perm[s : s + batch_size]
        yield X[ids], y[ids]

In [7]:
# Load the model
model = MLP(num_layers, train_images.shape[-1], hidden_dim, num_classes)
mlx.eval(model.parameters())

# Get a function which gives the loss and gradient of the
# loss with respect to the model's trainable parameters
loss_and_grad_fn = nn.value_and_grad(model, loss_fn)

# Instantiate the optimizer
optimizer = optim.SGD(learning_rate=learning_rate)

for e in range(num_epochs):
    for X, y in batch_iterate(batch_size, train_images, train_labels):
        loss, grads = loss_and_grad_fn(model, X, y)

        # Update the optimizer state and model parameters
        # in a single call
        optimizer.update(model, grads)

        # Force a graph evaluation
        mlx.eval(model.parameters(), optimizer.state)

    accuracy = eval_fn(model, test_images, test_labels)
    print(f"Epoch {e}: Test accuracy {accuracy.item():.3f}")

Epoch 0: Test accuracy 0.879
Epoch 1: Test accuracy 0.904
Epoch 2: Test accuracy 0.917
Epoch 3: Test accuracy 0.930
Epoch 4: Test accuracy 0.934
Epoch 5: Test accuracy 0.940
Epoch 6: Test accuracy 0.946
Epoch 7: Test accuracy 0.950
Epoch 8: Test accuracy 0.953
Epoch 9: Test accuracy 0.956
